In [1]:
import numpy as np
from tensorflow.keras import datasets

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [51]:
class Conv2D: # 卷积层
    def __init__(self, filter_n, filter_size):
        self.filter_n = filter_n # 滤波器/卷积核数量
        self.filter_size = filter_size # 滤波器/卷积核大小
        self.filters = np.random.randn(filter_n, filter_size, filter_size) / 9 # filter_n个滤波器/卷积核

    def generateRegions(self, input): # 生成区块们
        h, w = input.shape

        for i in range(h-self.filter_size+1):
            for j in range(w-self.filter_size+1):
                region = input[i:i+self.filter_size, j:j+self.filter_size]
                yield region, i, j

    def forward(self, input): # 前向传播
        h, w = input.shape
        output = np.zeros((h-self.filter_size+1, w-self.filter_size+1, self.filter_n))

        for region, i, j in self.generateRegions(input):
            output[i, j] = np.sum(region * self.filters, axis = (1, 2)) 

        self.input = input
        return output

    def backprop(self, d_L_d_output, alpha): # 反向传播
        d_L_d_filters = np.zeros(self.filters.shape)

        for region, i, j in self.generateRegions(self.input):
          for f in range(self.filter_n):
            d_L_d_filters[f] += d_L_d_output[i, j, f] * region

        self.filters -= alpha / (d_L_d_output.shape[0] * d_L_d_output.shape[1]) * d_L_d_filters # 更新卷积核们

In [52]:
class MaxPooling2D: # 池化层/采样层/汇聚层
    def generateRegions(self, input): # 生成区块们
        h, w, filter_n = input.shape

        for i in range(h // 2):
            for j in range(w // 2):
                region = input[i*2:i*2+2, j*2:j*2+2]
                yield region, i, j

    def forward(self, input): # 前向传播
        h, w, filter_n = input.shape
        output = np.zeros((h//2, w//2, filter_n))

        for region, i, j in self.generateRegions(input):
            output[i, j] = np.amax(region, axis = (0, 1)) 
        
        self.input = input
        self.output = output

        return output

    def backprop(self, d_L_d_output): # 反向传播
      d_L_d_input = np.zeros(self.input.shape)

      for region, i, j in self.generateRegions(self.input):
        h, w, filter_n = region.shape
        maxs = np.amax(region, axis=(0, 1)) # filter_n个2*2小区块每个中最大的那一个们

        for i2 in range(h):
          for j2 in range(w):
            for f2 in range(filter_n):
              if(region[i2, j2, f2] == maxs[f2]):
                d_L_d_input[i*2+i2, j*2+j2, f2] = d_L_d_output[i, j, f2]

      return d_L_d_input

In [53]:
class Softmax: # 全连接层
  def __init__(self, input_len, class_n):
    self.weights = np.random.randn(input_len+1, class_n) / (input_len+1) # 增广权向量
    self.class_n = class_n # 类别数

  def forward(self, input): # 前向传播
    x = np.append(input.flatten(), 1) # 增广特征向量
    fs = np.dot(x, self.weights) # f = wx
    exps = np.exp(fs)
    ps = exps / np.sum(exps, axis=0) # 属于各个类的概率

    self.input = input
    self.x = x
    self.ps = ps

    return ps

  def backprop(self, label, alpha): # 反向传播
    y = np.zeros(self.class_n)
    y[label] = 1

    d_L_d_x = - 1/self.class_n * self.weights[:-1] @ (y - self.ps) # 损失函数对输入x=input.flatten的导数
    self.weights += alpha * np.mat(self.x).T @ np.mat(y - self.ps) # SGD更新权重

    return d_L_d_x.reshape(self.input.shape)

In [61]:
conv = Conv2D(8, 3)
maxpool = MaxPooling2D()
dense = Softmax(13*13*8, 10)

In [62]:
def forward(image, label): # 前向传播
  output = conv.forward((image/255)-0.5)
  output = maxpool.forward(output)
  output = dense.forward(output)

  loss = -np.log(output[label]) # 损失
  acc = 1 if np.argmax(output) == label else 0 # 是否分类正确

  return output, loss, acc

def backprop(label, alpha): # 反向传播
  d_L = dense.backprop(label, alpha)
  d_L = maxpool.backprop(d_L)
  conv.backprop(d_L, alpha)

def train(image, label, alpha = .05): # 拿一个label类的image训练一次
  out, loss, acc = forward(image, label)
  backprop(label, alpha)
  return loss, acc

In [63]:
loss = 0
correct_n = 0

for epoch in range(3):
  print('--- Epoch %d ---' % (epoch + 1))

  for i, (im, label) in enumerate(zip(train_images[:1000], train_labels[:1000])): # 拿train_images[:1000]里的image们训练
    if(i%100 == 99):
      print('[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
        (i + 1, loss / 100, correct_n))
      loss = 0
      correct_n = 0

    lo, acc = train(im, label) # 拿一个label类的image训练一次
    loss += lo
    correct_n += acc


--- Epoch 1 ---
[Step 100] Past 100 steps: Average Loss 1.892 | Accuracy: 36%
[Step 200] Past 100 steps: Average Loss 1.264 | Accuracy: 65%
[Step 300] Past 100 steps: Average Loss 0.956 | Accuracy: 74%
[Step 400] Past 100 steps: Average Loss 0.720 | Accuracy: 80%
[Step 500] Past 100 steps: Average Loss 0.744 | Accuracy: 81%
[Step 600] Past 100 steps: Average Loss 0.843 | Accuracy: 72%
[Step 700] Past 100 steps: Average Loss 0.755 | Accuracy: 78%
[Step 800] Past 100 steps: Average Loss 0.548 | Accuracy: 84%
[Step 900] Past 100 steps: Average Loss 0.665 | Accuracy: 81%
[Step 1000] Past 100 steps: Average Loss 0.690 | Accuracy: 81%
--- Epoch 2 ---
[Step 100] Past 100 steps: Average Loss 0.493 | Accuracy: 86%
[Step 200] Past 100 steps: Average Loss 0.477 | Accuracy: 85%
[Step 300] Past 100 steps: Average Loss 0.484 | Accuracy: 86%
[Step 400] Past 100 steps: Average Loss 0.289 | Accuracy: 92%
[Step 500] Past 100 steps: Average Loss 0.464 | Accuracy: 85%
[Step 600] Past 100 steps: Average Lo

In [64]:
# Test the CNN
print('\n--- Testing the CNN ---')
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

num_tests = len(test_images)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)


--- Testing the CNN ---
Test Loss: 0.5040411338286891
Test Accuracy: 0.8364
